In [12]:
##############
#K. David Roell CFPB 7/12/16
#expands data selection for quality edits to include context for developing statistical approaches
#will attemtp to segment mortgage market participants by loan activity by:
#volume/count of lending, income of applicants, securitization, property type and other factors
##############

import json
import os
import pandas as pd
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
get_ipython().magic(u'matplotlib inline')
matplotlib.style.use('ggplot')

with open('quality_sql.json') as f: #FIXME change to quality_sql.json and refactor the rest of code
    edit_sql = json.load(f)
    
#parameter format for local use #consider changing hmdamaster to roellk if db changes cause a fail
params = {
'dbname':'hmdamaster',
'user':'roellk',
'password':'',
'host':'localhost',
}

try:
    conn = psycopg2.connect(**params)
    cur = conn.cursor()
    print("i'm connected")

except psycopg2.Error as e: #if database connection results in an error print the following
    print("I am unable to connect to the database: ", e)

i'm connected


In [8]:
#get distributions of values for fields
def field_dist_geo(field1=None, field2=None, table=None, geo_level=None, prop_type=None, loan_purp=None, loan_type=None, occupancy=None, lien=None):
    """generates a sql statement used to pull values and associated counts for the listed variable subject to listed filters"""
    #FIXME use a list instead of prop_type, loan_purp, loan_type
    for key, value in kwargs.items():
        
    if geo_level == 'msa':
        geo_sql = 'msa,'
    elif geo_level == 'state':
        geo_sql = 'state,'
    elif geo_level == 'county':
        geo_sql = 'CONCAT(state,county),'
    elif geo_level == 'tract':
        geo_sql = 'CONCAT(state,county,tract),'
    else:
        geo_sql = ''
    
    dist_sql = """SELECT DISTINCT year, {geo_sql} {field1}, {field2}
    FROM {table} """.format(table=table, field1=field1, field2=field2, geo_sql=geo_sql) 
    #FIXME make flexible to acommodate lack of geography
    #FIXME convert named params to None and raise value error 
    #removing the bulk of quesitonable data in exploration by removing HUD
    where_sql = """WHERE agency != '7' AND property_type IN ('{prop_type}') AND loan_purpose IN ('{loan_purp}') 
    AND loan_type IN ({loanCodes}) AND occupancy='{occupancy}' AND lien = '{lien}' AND income NOT ILIKE '%NA%' AND amount NOT ILIKE 
    '%NA%' """.format(prop_type=prop_type, loan_purp=loan_purp,  lien=lien, occupancy=occupancy, loanCodes=loanCodes)

    group_sql = """GROUP BY year, {geo_group} {field1}, {field2}""".format(field1=field1, field2=field2, geo_group=geo_sql)
    order_sql = """ ORDER by {field1}, {field2} ASC """.format(field1=field1, field2=field2)
    return_sql = dist_sql + where_sql + group_sql + order_sql
    print(return_sql)
    return return_sql
#test 90000451965

In [23]:
def get_field_dist(year=None, field1=None, field2=None, geo_level='', prop_type=None, loan_purp=None, loan_type=None, occupancy=None, lien=None):
    """executes SQL to get distributions of data by selected geography for the stated field for multiple years of hmda"""
    first = True
    while year > 2003:
        table = 'hmdalar' + str(year)
        print(table)
        cur.execute(field_dist_geo(table=table, field1=field1, field2=field2, geo_level=geo_level, prop_type=prop_type,  loan_purp=loan_purp, occupancy=occupancy, lien=lien))        
        data_df = pd.DataFrame(cur.fetchall())

        col_names = [desc[0] for desc in cur.description]
        if len(data_df.columns) == len(col_names):
            print('setting column names')
            data_df.columns = col_names

        if first == True and len(data_df.columns) > 0:
            dist_df = data_df.copy()
            first = False

        elif first == False and len(data_df.columns) > 0:
            print('merging dataframes')
            dist_df=pd.concat([dist_df, data_df])

        else:
            print('no data returned from query')

        print(dist_df.tail())
        path = 'data/'
        if not os.path.exists(path):
            os.makedirs(path)

        year -= 1
    dist_df.to_csv(path + '{field1}_to_{field2}_dist_{geo_level}_{loanType}.csv'.format(field1=field1, field2=field2, geo_level=geo_level, loanType=loanType),index=False)

In [24]:
loanType = 'RHS'
if loanType == 'all':
    loanCodes = "'1', '2', '3', '4'"
elif loanType == 'CONV':
    loanCodes = "'1'"
elif loanType == 'VA':
    loanCodes = "'2'"
elif loanType == 'FHA':
    loanCodes = "'3'"
elif loanType == 'RHS':
    loanCodes = "'4'"
    

In [ ]:
#FIXME convert to **kwargs
kwargs={'year':2014,
        'field1':'income',
        'field2':'amount',
        'loan_type': loanCodes,
        'loan_purp':'1',
        'prop_type':'1',
        'occupancy':'1',
        'lien':'1'}
#get_field_dist(year=2014, field1='income', field2='amount', loan_type=loanCodes, loan_purp='1', prop_type='1',  occupancy='1', lien='1')
get_field_dist(**kwargs)

hmdalar2014
SELECT DISTINCT year,  income, amount
    FROM hmdalar2014 WHERE agency != '7' AND property_type IN ('1') AND loan_purpose IN ('1') 
    AND loan_type IN ('4') AND occupancy='1' AND lien = '1' AND income NOT ILIKE '%NA%' AND amount NOT ILIKE 
    '%NA%' GROUP BY year,  income, amount ORDER by income, amount ASC 
